# Auto Poster Generation
## 当前任务
### 打分器（实际上可看作二分类问题）
#### 1. 输入
- 正样本：已有的海报图像，label为1
- 负样本：在现有海报图像的基础上随机搭配，label为0

#### 2. 网络结构
- 可以用现有的卷积基模型，可能需要fine-tune
- 也可以自己构造一个简单的模型（尝试）
- 对于卷积基提取的特征，后面接上Flatten和Dense层，最后做一个二分类

#### 3. 输出
- 输出的概率值既可以看作是打分器的分数

In [ ]:
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
    
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

In [ ]:
standard_width = 200
standard_height = 280

### 读取数据集并进行预处理

In [ ]:
import glob
import numpy as np
from PIL import Image

poster_positive = glob.glob('./data/poster_positive/*.png')
poster_negative = glob.glob('./data/poster_negative/*.png')
# poster_positive += glob.glob('./data/poster_positive_aug/*.png')
# poster_negative += glob.glob('./data/poster_negative_aug/*.png')

np.random.shuffle(poster_positive)
np.random.shuffle(poster_negative)
poster_positive_num = len(poster_positive)
poster_negative_num = len(poster_negative)

print("poster positive num: " + str(poster_positive_num))
print("poster negative num: " + str(poster_negative_num))

## 使用RGB图像训练-单进程

In [ ]:
import random

num_train_positive = 297
# num_train_negative = 1352
num_train_negative = 297
num_validation_positive = 36
# num_validation_negative = 169
num_validation_negative = 36
num_test_positive = 36
# num_test_negative = 169
num_test_negative = 36

X_train_s = np.empty((num_train_positive + num_train_negative, standard_height, standard_width, 3))
Y_train_s = np.empty((num_train_positive + num_train_negative, 1))

X_validation_s = np.empty((num_validation_positive + num_validation_negative, standard_height, standard_width, 3))
Y_validation_s = np.empty((num_validation_positive + num_validation_negative, 1))

X_test_s = np.empty((num_test_positive + num_test_negative, standard_height, standard_width, 3))
Y_test_s = np.empty((num_test_positive + num_test_negative, 1))

for i in range(num_train_positive):
    im = Image.open(poster_positive[i])
    X_train_s[i] = np.asarray(im.convert('RGB'), dtype='float64') / 255.0  
    Y_train_s[i] = 1
    
for i in range(num_train_negative):
    im = Image.open(poster_negative[i])
    X_train_s[num_train_positive + i] = np.asarray(im.convert('RGB'), dtype='float64') /255.0
    Y_train_s[num_train_positive + i] = 0
    
# index = [i for i in range(len(X_train_s))]
# random.shuffle(index)
# X_train_s = X_train_s[index]
# Y_train_s = Y_train_s[index]

for i in range(num_validation_positive):
    im = Image.open(poster_positive[num_train_positive + i])
    X_validation_s[i] = np.asarray(im.convert('RGB'), dtype='float64') / 255.0  
    Y_validation_s[i] = 1
    
for i in range(num_validation_negative):
    im = Image.open(poster_negative[num_train_negative + i])
    X_validation_s[num_validation_positive + i] = np.asarray(im.convert('RGB'), dtype='float64') /255.0
    Y_validation_s[num_validation_positive + i] = 0
    
for i in range(num_test_positive):
    im = Image.open(poster_positive[num_train_positive + num_validation_positive + i])
    X_test_s[i] = np.asarray(im.convert('RGB'), dtype='float64') / 255.0  
    Y_test_s[i] = 1
    
for i in range(num_test_negative):
    im = Image.open(poster_negative[num_train_negative + num_validation_negative + i])
    X_test_s[num_test_positive + i] = np.asarray(im.convert('RGB'), dtype='float64') /255.0
    Y_test_s[num_test_positive + i] = 0

## 使用RGB图像训练-多进程

In [ ]:
import random
from multiprocessing import Pool
from multiprocessing import sharedctypes
import import_ipynb
from imgAugmentation import ImageProcess
import ctypes

debug = False

num_train_positive = 1
num_train_negative = 1

num_validation_positive = 200
num_validation_negative = 200

num_test_positive = 200
num_test_negative = 200

# tuple(number, height, width, dimension)

X_train = np.ctypeslib.as_ctypes(np.empty((num_train_positive + num_train_negative, standard_height, standard_width, 3)))
Y_train = np.ctypeslib.as_ctypes(np.empty((num_train_positive + num_train_negative, 1)))

X_validation = np.ctypeslib.as_ctypes(np.empty((num_validation_positive + num_validation_negative, standard_height, standard_width, 3)))
Y_validation = np.ctypeslib.as_ctypes(np.empty((num_validation_positive + num_validation_negative, 1)))

X_test = np.ctypeslib.as_ctypes(np.empty((num_test_positive + num_test_negative, standard_height, standard_width, 3)))
Y_test = np.ctypeslib.as_ctypes(np.empty((num_test_positive + num_test_negative, 1)))

# create Shared_memory of each set
X_train_sh = sharedctypes.RawArray(X_train._type_, X_train)
Y_train_sh = sharedctypes.RawArray(Y_train._type_, Y_train)

X_validation_sh = sharedctypes.RawArray(X_validation._type_, X_validation)
Y_validation_sh = sharedctypes.RawArray(Y_validation._type_, Y_validation)

X_test_sh = sharedctypes.RawArray(X_test._type_, X_test)
Y_test_sh = sharedctypes.RawArray(Y_test._type_, Y_test)


print('begin to process')

def trainSetPositive(i):
    try:
        im = Image.open(poster_positive[i]).convert('RGB')
        X_train_sh[i] = np.ctypeslib.as_ctypes(np.array(im) / 255.)
        Y_train_sh[i] = np.ctypeslib.as_ctypes(np.array([1.]))
        if debug:
            print('finish train positive', i)
    except Exception as e:
        print(e)

def trainSetNegative(i):
    try:
        im = Image.open(poster_negative[i]).convert('RGB')
        X_train_sh[num_train_positive + i] = np.ctypeslib.as_ctypes(np.array(im) / 255.)
        Y_train_sh[num_train_positive + i] = np.ctypeslib.as_ctypes(np.array([0.]))
        if debug:
            print('finish train negative', i)
    except Exception as e:
        print(e)
    
def validationSetPositive(i):
    im = Image.open(poster_positive[num_train_positive + i]).convert('RGB')
    X_validation_sh[i] = np.ctypeslib.as_ctypes(np.array(im) / 255.)
    Y_validation_sh[i] = np.ctypeslib.as_ctypes(np.array([1.]))
    if debug:
        print('finish validation positive', i)
    
def validationSetNegative(i):
    im = Image.open(poster_negative[num_train_negative + i]).convert('RGB')
    X_validation_sh[num_validation_positive + i] = np.ctypeslib.as_ctypes(np.array(im) / 255.)
    Y_validation_sh[num_validation_positive + i] = np.ctypeslib.as_ctypes(np.array([0.]))
    if debug:
        print('finish validation negative', i)
    
def testSetPositive(i):
    im = Image.open(poster_positive[num_train_positive + num_validation_positive + i]).convert('RGB')
    X_test_sh[i] = np.ctypeslib.as_ctypes(np.array(im) / 255.)
    Y_test_sh[i] = np.ctypeslib.as_ctypes(np.array([1.]))
    if debug:
        print('finish test positive', i)
    
def testSetNegative(i):
    im = Image.open(poster_negative[num_train_negative + num_validation_negative + i]).convert('RGB')
    X_test_sh[num_test_positive + i] = np.ctypeslib.as_ctypes(np.array(im) / 255.)
    Y_test_sh[num_test_positive + i] = np.ctypeslib.as_ctypes(np.array([0.]))
    if debug:
        print('finish test negative', i)
    
# initialize the process pool
## ATTENTION: pool must be initialized after function
pool = Pool()


# make train set

for i in range(num_train_positive):
    pool.apply_async(trainSetPositive, args=(i,))
for i in range(num_train_negative):
    pool.apply_async(trainSetNegative, args=(i,))

# convert Ctype to numpy
X_train = np.ctypeslib.as_array(X_train_sh)
Y_train = np.ctypeslib.as_array(Y_train_sh)
    
# # shuffle the whole train set
# zipped = list(zip(X_train, Y_train))
# np.random.shuffle(zipped)
# X_train[:], Y_train[:] = zip(*zipped)

# another shuffle method
# index = [i for i in range(len(X_train))]
# random.shuffle(index)
# X_train = X_train[index]
# Y_train = Y_train[index]


print(np.array(X_train).shape, np.array(Y_train).shape)
assert len(X_train) == len(Y_train)


# make validation set

for i in range(num_validation_positive):
    pool.apply_async(validationSetPositive, args=(i,))
    
for i in range(num_validation_negative):
    pool.apply_async(validationSetNegative, args=(i,))

# convert Ctype to numpy
X_validation = np.ctypeslib.as_array(X_validation_sh)
Y_validation = np.ctypeslib.as_array(Y_validation_sh)

print(np.array(X_validation).shape, np.array(Y_validation).shape)
assert len(X_validation) == len(Y_validation)


# make test set
for i in range(num_test_positive):
    pool.apply_async(testSetPositive, args=(i,))

for i in range(num_test_negative):
    pool.apply_async(testSetNegative, args=(i,))

# convert Ctype to numpy
X_test = np.ctypeslib.as_array(X_test_sh)
Y_test = np.ctypeslib.as_array(Y_test_sh)

print(np.array(X_test).shape, np.array(Y_test).shape)
assert len(X_test) == len(Y_test)

pool.close()
pool.join()

## 使用YUV图像训练

In [ ]:
import random
from multiprocessing import Pool
from multiprocessing import sharedctypes
import import_ipynb
from imgAugmentation import ImageProcess
import ctypes

num_train_positive = 6800
num_train_negative = 6800

num_validation_positive = 200
num_validation_negative = 200

num_test_positive = 200
num_test_negative = 200

# tuple(number, height, width, dimension)

X_train = np.ctypeslib.as_ctypes(np.empty((num_train_positive + num_train_negative, standard_height, standard_width, 3)))
Y_train = np.ctypeslib.as_ctypes(np.empty((num_train_positive + num_train_negative, 1)))

X_validation = np.ctypeslib.as_ctypes(np.empty((num_validation_positive + num_validation_negative, standard_height, standard_width, 3)))
Y_validation = np.ctypeslib.as_ctypes(np.empty((num_validation_positive + num_validation_negative, 1)))

X_test = np.ctypeslib.as_ctypes(np.empty((num_test_positive + num_test_negative, standard_height, standard_width, 3)))
Y_test = np.ctypeslib.as_ctypes(np.empty((num_test_positive + num_test_negative, 1)))

# create Shared_memory of each set
X_train_sh = sharedctypes.RawArray(X_train._type_, X_train)
Y_train_sh = sharedctypes.RawArray(Y_train._type_, Y_train)

X_validation_sh = sharedctypes.RawArray(X_validation._type_, X_validation)
Y_validation_sh = sharedctypes.RawArray(Y_validation._type_, Y_validation)

X_test_sh = sharedctypes.RawArray(X_test._type_, X_test)
Y_test_sh = sharedctypes.RawArray(Y_test._type_, Y_test)


print('begin to process')

def trainSetPositive(i):
    im = Image.open(poster_positive[i])
    im = ImageProcess.rgb_to_yuv(im.convert('RGB'))
    im = np.array(im[:,:,:])
    X_train_sh[i] = np.ctypeslib.as_ctypes(np.asarray(im, dtype='float64'))
    Y_train_sh[i] = np.ctypeslib.as_ctypes(np.array([1.]))
    
def trainSetNegative(i):
    im = Image.open(poster_negative[i])
    im = ImageProcess.rgb_to_yuv(im.convert('RGB'))
    im = np.array(im[:,:,:])
    X_train_sh[num_train_positive + i] = np.ctypeslib.as_ctypes(np.asarray(im, dtype='float64'))
    Y_train_sh[num_train_positive + i] = np.ctypeslib.as_ctypes(np.array([0.]))
    
def validationSetPositive(i):
    im = Image.open(poster_positive[num_train_positive + i])
    im = ImageProcess.rgb_to_yuv(im.convert('RGB'))
    im = np.array(im[:,:,:])
    X_validation_sh[i] = np.ctypeslib.as_ctypes(np.asarray(im, dtype='float64'))
    Y_validation_sh[i] = np.ctypeslib.as_ctypes(np.array([1.]))
    
def validationSetNegative(i):
    im = Image.open(poster_negative[num_train_negative + i])
    im = ImageProcess.rgb_to_yuv(im.convert('RGB'))
    im = np.array(im[:,:,:])
    X_validation_sh[num_validation_positive + i] = np.ctypeslib.as_ctypes(np.asarray(im, dtype='float64'))
    Y_validation_sh[num_validation_positive + i] = np.ctypeslib.as_ctypes(np.array([0.]))
    
def testSetPositive(i):
    im = Image.open(poster_positive[num_train_positive + num_validation_positive + i])
    im = ImageProcess.rgb_to_yuv(im.convert('RGB'))
    im = np.array(im[:,:,:])
    X_test_sh[i] = np.ctypeslib.as_ctypes(np.asarray(im, dtype='float64'))
    Y_test_sh[i] = np.ctypeslib.as_ctypes(np.array([1.]))
    
def testSetNegative(i):
    im = Image.open(poster_negative[num_train_negative + num_validation_negative + i])
    im = ImageProcess.rgb_to_yuv(im.convert('RGB'))
    im = np.array(im[:,:,:])
    X_test_sh[num_test_positive + i] = np.ctypeslib.as_ctypes(np.asarray(im, dtype='float64'))
    Y_test_sh[num_test_positive + i] = np.ctypeslib.as_ctypes(np.array([0.]))
    
# initialize the process pool
## ATTENTION: pool must be initialized after function
pool = Pool()


# make train set

for i in range(num_train_positive):
    pool.apply_async(trainSetPositive, args=(i,))
for i in range(num_train_negative):
    pool.apply_async(trainSetNegative, args=(i,))

# convert Ctype to numpy
X_train = np.ctypeslib.as_array(X_train_sh)
Y_train = np.ctypeslib.as_array(Y_train_sh)
    
# shuffle the whole train set
zipped = list(zip(X_train, Y_train))
np.random.shuffle(zipped)
X_train[:], Y_train[:] = zip(*zipped)

print(np.array(X_train).shape, np.array(Y_train).shape)
assert len(X_train) == len(Y_train)

    
# make validation set

for i in range(num_validation_positive):
    pool.apply_async(validationSetPositive, args=(i,))
    
for i in range(num_validation_negative):
    pool.apply_async(validationSetNegative, args=(i,))

# convert Ctype to numpy
X_validation = np.ctypeslib.as_array(X_validation_sh)
Y_validation = np.ctypeslib.as_array(Y_validation_sh)

print(np.array(X_validation).shape, np.array(Y_validation).shape)
assert len(X_validation) == len(Y_validation)


# make test set
for i in range(num_test_positive):
    pool.apply_async(testSetPositive, args=(i,))

for i in range(num_test_negative):
    pool.apply_async(testSetNegative, args=(i,))

# convert Ctype to numpy
X_test = np.ctypeslib.as_array(X_test_sh)
Y_test = np.ctypeslib.as_array(Y_test_sh)

print(np.array(X_test).shape, np.array(Y_test).shape)
assert len(X_test) == len(Y_test)

pool.close()
pool.join()

### 尝试使用VGG16卷积基预训练模型

In [ ]:
# 将VGG16卷积基实例化
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(standard_height, standard_width, 3))

In [ ]:
conv_base.summary()

In [ ]:
# 在卷积基上添加一个密集链接分类器
from keras import models
from keras import layers
from keras import regularizers
from keras.models import load_model

# model = models.Sequential()
# model.add(conv_base)
# model.add(layers.Flatten())
# model.add(layers.Dense(256, activation='relu',
#                        kernel_initializer='glorot_normal', bias_initializer='zeros',
#                        kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)))
# model.add(layers.Dense(1,activation='sigmoid', kernel_initializer='glorot_normal', bias_initializer='zeros'))

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
# 冻结卷积基
conv_base.trainable = False
print(len(model.trainable_weights))

In [ ]:
# 编译模型
model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
print(Y_train.shape)

In [ ]:
# 训练模型
model_path = './model_save/vgg16_plain.h5'
if os.path.exists(model_path):
    model = load_model(model_path)
history = model.fit(X_train,
                    Y_train,
                    epochs=2,
                    batch_size=70,
                    validation_data=(X_validation, Y_validation))

# model.save(model_path)

In [ ]:
results = model.evaluate(X_test, Y_test)
print(results)

In [ ]:
model.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
def trainVisualization(hist):
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']
    acc = hist.history['acc']
    val_acc = hist.history['val_acc']

    # make a figure
    fig = plt.figure(figsize=(8,4))

    # subplot loss
    ax1 = fig.add_subplot(121)
    ax1.plot(loss,label='train_loss')
    ax1.plot(val_loss,label='val_loss')
    ax1.scatter(history.epoch, history.history['loss'], marker='*')
    ax1.scatter(history.epoch, history.history['val_loss'], marker='*')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.set_title('Loss on Training and Validation Data')
    ax1.legend(loc='upper right')

    # subplot acc
    ax2 = fig.add_subplot(122)
    ax2.plot(acc,label='train_acc')
    ax2.plot(val_acc,label='val_acc')
    ax2.scatter(history.epoch, history.history['acc'], marker='*')
    ax2.scatter(history.epoch, history.history['val_acc'])
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Accuracy  on Training and Validation Data')
    ax1.legend(loc='lower right')

trainVisualization(history)

### VGG16预训练模型效果不好，接下来重新训练一个以Xception为卷积基的网络

In [ ]:
from keras.applications import Xception
from keras import models
from keras import layers

conv_base_xception = Xception(include_top=False,
                              input_shape=(standard_height, standard_width, 3))

In [ ]:
conv_base_xception.summary()

In [ ]:
# 在xception卷积基上添加一个密集链接分类器

model_xception = models.Sequential()
model_xception.add(conv_base_xception)
model_xception.add(layers.Flatten())
model_xception.add(layers.Dense(256, activation='relu'))
model_xception.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
model_xception.summary()

In [ ]:
print(len(model_xception.trainable_weights))

In [ ]:
# 编译模型
model_xception.compile(optimizer='rmsprop',
                       loss='binary_crossentropy',
                       metrics=['accuracy']) 

In [ ]:
# 训练模型
from keras.backend import get_session

get_session().run(tf.global_variables_initializer())

history_xception = model_xception.fit(X_train,
                                      Y_train,
                                      epochs=20,
                                      batch_size=80,
                                      validation_data=(X_validation, Y_validation))

model_xception.save('scorer_xception.h5')

### 尝试加入style matrix(gram matrix)，这里还是使用VGG16，因为只包含卷积和池化基本操作

In [ ]:
# 将VGG16卷积基实例化，这次不含参数
from keras.applications import VGG16

conv_base_vgg16 = VGG16(include_top=False,
                        input_shape=(standard_height, standard_width, 3))

In [ ]:
conv_base_vgg16.summary()

In [ ]:
# style matrix(gram matrix)

from keras import backend as K

# def gram_matrix(A):
#     """
#     Argument:
#     A -- matrix of shape (n_C, n_H*n_W)
    
#     Returns:
#     GA -- Gram matrix of A, of shape (n_C, n_C)
#     """
    
#     GA = K.dot(A, K.transpose(A))
    
#     return GA

def compute_layer_style(a_S):
#     """
#     Arguments:
#     a_S -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing style of the image S 

#     Returns: 
#     GS -- Gram matrix of S, of shape (n_C, n_C)        
#     """
    
    GS = K.batch_dot(a_S, a_S, axes=[1, 1])
    
    return GS

In [ ]:
# 构建网络

from keras.models import Model
from keras import layers
from keras import Input

def model_vgg16_style(input_shape):
    
    X_input = Input(input_shape)
    
    X = conv_base_vgg16(X_input)
    
    m, n_H, n_W, n_C = X.get_shape().as_list()
    
    X = layers.core.Reshape([n_H*n_W, n_C])(X)
    
    X = layers.core.Lambda(compute_layer_style)(X)
    
    X = layers.Flatten()(X)
    
    X = layers.Dense(256, activation='relu')(X)
    
    X = layers.Dense(256, activation='relu')(X)
    
    X = layers.Dense(1, activation='sigmoid')(X)
    
    model_vgg16_style = Model(X_input, X)
    
    return model_vgg16_style

In [ ]:
model_vgg16_style = model_vgg16_style((standard_height, standard_width, 3))

In [ ]:
model_vgg16_style.summary()

In [ ]:
# 编译模型
model_vgg16_style.compile(optimizer='rmsprop',
                          loss='binary_crossentropy',
                          metrics=['accuracy'])

In [ ]:
# 训练模型
from keras.backend import get_session

get_session().run(tf.global_variables_initializer())

history_vgg16_style = model_vgg16_style.fit(X_train,
                                            Y_train,
                                            epochs=5,
                                            batch_size=16,
                                            validation_data=(X_validation, Y_validation))

model_vgg16_style.save('scorer_vgg16_style.h5')

### 可能海报提取成高维特征后，就会不收敛，下面尝试搭建一个只带一次卷积池化的简单网络

In [ ]:
from keras.models import Model
from keras import layers
from keras import Input

def Simple_Model(input_shape):
    
    X_input = Input(input_shape)
        
    X = layers.ZeroPadding2D((3, 3))(X_input)
        
    X = layers.Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = layers.BatchNormalization(axis = 3, name = 'bn0')(X)
    X = layers.Activation('relu')(X)
        
    X = layers.MaxPooling2D((2, 2), name='max_pool')(X)
        
    X = layers.Flatten()(X)
    X = layers.Dense(1, activation='sigmoid', name='fc')(X)
        
    simple_model = Model(inputs = X_input, outputs = X, name='Simple_Model')
    
    
    return simple_model

In [ ]:
simple_model = Simple_Model((standard_height, standard_width, 3))

In [ ]:
simple_model.summary()

In [ ]:
simple_model.compile(optimizer = "Adam",
                     loss = "binary_crossentropy",
                     metrics = ["accuracy"])

In [ ]:
history_simple = simple_model.fit(X_train,
                                  Y_train,
                                  epochs=5,
                                  batch_size=16,
                                  validation_data=(X_validation, Y_validation))

### 尝试只用全连接层

In [ ]:
from keras.models import Model
from keras import layers
from keras import Input

def dense_model(input_shape):
    
    X_input = Input(input_shape)
    
#     m, n_H, n_W, n_C = X_input.get_shape().as_list()
    
#     X = layers.core.Reshape([n_H*n_W, n_C])(X_input)
    
#     X = layers.core.Lambda(compute_layer_style)(X)
    
    X = layers.Flatten()(X_input)
    
    X = layers.Dense(512, activation='relu')(X)
    
    X = layers.Dense(512, activation='relu')(X)
    
    X = layers.Dense(1, activation='sigmoid')(X)
    
    dense_model = Model(inputs = X_input, outputs = X)
    
    return dense_model

In [ ]:
dense_model = dense_model((standard_height, standard_width, 3))

In [ ]:
dense_model.summary()

In [ ]:
dense_model.compile(optimizer = "Adam",
                    loss = "binary_crossentropy",
                    metrics = ["accuracy"])

In [ ]:
dense_simple = dense_model.fit(X_train,
                               Y_train,
                               epochs=5,
                               batch_size=16,
                               validation_data=(X_validation, Y_validation))